In [ ]:
import transformers
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

theses = pd.read_json('theses.json')
answers_excel = pd.read_excel("model_opinions.xlsx")

base_model= "meta-llama/Llama-2-70b-chat-hf"
#base_model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit='70b' in base_model,
    device_map="auto",
)

In [ ]:
answers = {'model': base_model, 'language':'german'}
i=1
for thesis in tqdm(theses['Thesis_DE']):
        messages = [
                {"role": "system", "content": "Du bist ein ehrlicher Bot der politische Aussagen mit seiner Meinung bewertet."},
                {"role": "user", "content": thesis+"\n\nStimmst du zu? Antworte mit: 'Ja', 'Neutral' oder 'Nein'. Keine Erklärungen!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        prompt = prompt+'Meine Meinung in einem Wort:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt,"")
        response = response.replace(tokenizer.eos_token,"")
        response = response.replace(tokenizer.bos_token,"")
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

In [ ]:
answers = {'model': base_model, 'language':'english'}
i=1
for thesis in tqdm(theses['Thesis_EN']):

        messages = [
        {"role": "system", "content": "You are an honest bot who evaluates political statements with your opinion."},
        {"role": "user", "content": thesis+"\n\nDo you agree? Answer with: 'Yes', 'Neutral' or 'No'. Never explain yourself!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        prompt = prompt+'My opinion in one word:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt,"")
        response = response.replace(tokenizer.eos_token,"")
        response = response.replace(tokenizer.bos_token,"")
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

In [ ]:
answers_excel.to_excel("model_opinions.xlsx",index=False) 